In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from utils import *
import math as m
from tensorflow.keras.initializers import RandomNormal, RandomUniform, Constant, Zeros
from collections import defaultdict
from sklearn.preprocessing import *
from IPython.display import clear_output
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from tensorflow.keras import regularizers
from sklearn.model_selection import StratifiedKFold
from scipy.io import loadmat
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def yes(a, b):
    for i in a:
        if i not in b:
            return False
    return True

In [ ]:
def get_data(path_dir, paths, freq):
    X, Y = [], []
    included = []
    
    df = read_file(path)
    control = "C" in path[path.rfind('\\')+2:]
    name = path[path.rfind('\\')+1:]
#     print(name)
#     chans = ['f7', 'c3', 'p3', 'f3', 'fp2', 'p4', 'f8', 'c4', 'f4', 'fp1', 'o2', 'fz', 'o1', 'cz', 'a1']
#     if not yes(chans, df.columns):
#         return [], [], []
#     data = df[chans]
    if name in d:
        print(name, d[name])
        data = df[d[name]]
    else:
        return [], [], []
#     if data.shape[1] != len(chans):
#         return [], [], []
    included.append(name)
    f = get_freq(df)
    if f != freq:
        data = resample(data, df['Time'], f, freq)
    X.append(np.array(data))
    Y.append(0 if control else 1)
    return X, Y, included

In [ ]:
def transform(X):
    res = []
    for freq, time in zip(options['freqs'], options['d_units']):
        x = resample(X, np.array([i / f for i in range(X.shape[0])]), f, freq)
        assert(x.shape[0] >= time)
        res.append(x[:time])
    return res

In [ ]:
def to_train_samples(samples, group, inds, units=800):
    X = []
    Y = []
    
    for j in inds:
        sample = np.asarray(samples[j])
        if options['normalize']:
            sample = StandardScaler().fit_transform(sample)
        if options['min_max']:
#             sample = MinMaxScaler((options['min'], options['max'])).fit_transform(sample)
            sample = sample / sample.max()
#             sample = (sample-sample.min())/(sample.max()-sample.min())
#             sample = (sample*(options['max']-options['min']))+options['min']
        if options['smooth'] != 0:
            sample = butter_lowpass_filter(sample, options['smooth'], f, options['order'])
        offset = np.random.randint(units)
        for i in range(units+offset, sample.shape[0], options['offset']):
            #X.append(sample[:, i-time:i])
            X.append(transform(sample[i-units:i, :]))
            Y.append(1 if group[j] else 0)
        #sample = butter_lowpass_filter(sample, 50, 200, 2)
        #print(sample.std(axis=0), included[j])
    res_X = []
    for i in range(len(X[0])):
        res_X.append(np.array([x[i] for x in X]))
    inds = random.sample([i for i in range(len(X))], k=min(len(X), 152))
    return res_X, np.expand_dims(Y, -1)

In [ ]:
class SyncNetLayer(tf.keras.layers.Layer):
    def __init__(self, options, length, i, **kwargs):
        super().__init__(**kwargs)
        self.options = options
        self.Nt = length
        self.pool_size = options['pool_size'][i]
        self.stride = options['stride'][i]
        self.upper_omega = options['upper_omega'][i]
        self.lower_omega = options['lower_omega'][i]
        self.freq = options['freqs'][i]
        
    def build(self, batch_input_shape):
        self.b = self.add_weight(name="b", 
                                 shape=[1,1,self.options['C'], self.options['K']], 
#                                  initializer='glorot_normal',
                                 initializer=RandomNormal(mean=0, stddev=0.01),
                                 regularizer = tf.keras.regularizers.l2(options['lambda_b']))
        pi = m.pi
        self.omega = self.add_weight(name="omega", 
                                     shape=[1,1,1,self.options['K']], 
#                                      initializer='glorot_normal',
                                     initializer=RandomUniform(minval=self.lower_omega, maxval=2*pi*self.upper_omega/self.freq),
                                    regularizer = tf.keras.regularizers.l2(options['lambda_omega']))
#                                      initializer='glorot_uniform')
#                                      initializer=RandomUniform(minval=0, maxval=2*pi*self.omega/self.freq))
        
        self.phi = self.add_weight(name="phi", 
                                  shape=[1,1,self.options['C'], self.options['K']],
#                                   initializer='glorot_normal',
                                   initializer=Zeros(),
                                  regularizer = tf.keras.regularizers.l2(options['lambda_phi']))
#                                    initializer='glorot_uniform')
#                                    initializer=RandomNormal(mean=0, stddev=0.05))
        
        self.beta = self.add_weight(name="beta",
                                   shape=[1,1,1,self.options['K']],
#                                    initializer='glorot_normal',
                                    initializer=RandomNormal(mean=0, stddev=0.05),
                                   regularizer = tf.keras.regularizers.l2(options['lambda_beta']),
                                   constraint = tf.keras.constraints.NonNeg())
#                                     initializer='glorot_uniform')
#                                     initializer=RandomNormal(mean=0, stddev=0.05))
        
#         self.bias = self.add_weight(name="bias",
#                                    shape = [1])
        
        self.t= tf.cast(tf.reshape(range(-self.Nt//2,self.Nt//2),[1,self.Nt,1,1]), dtype=tf.float32)
        
        super().build(batch_input_shape)
        
    def call(self, X):
        X = tf.expand_dims(X, axis=1)
        W_osc = self.b * tf.cos(self.t*self.omega + self.phi)
        W_decay = tf.exp(-self.t**2*self.beta)
        W = W_osc * W_decay
        
        h_conv = tf.nn.relu(tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME'))
#         h_conv = tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')
        self.h_pool = tf.nn.avg_pool(h_conv, ksize=[1,1,self.pool_size, 1],  strides=[1, 1, self.stride, 1], padding='SAME') #+ self.bias
        
        return self.h_pool
    

In [ ]:
class MyDropout(tf.keras.layers.Layer):
    def __init__(self, r, num, **kwargs):
        super().__init__(**kwargs)
        self.dropout_rate = r
        self.num = num
        
    def call(self, X):
        res = None
        if np.random.rand() < self.dropout_rate:
            res= tf.concat([X[:-self.num], tf.zeros_like(X[-self.num:])], axis=0)
        else:
            res= tf.concat([tf.zeros_like(X[:-self.num]), X[-self.num:]], axis=0)
        return res

In [ ]:
# class TransformLayer(tf.keras.layers.Layer):
#     def __init__(self, options, **kwargs):
#         super().__init__(kwargs)
#         self.options = options
        
#     def call(self, X):
        

In [ ]:
def get_eeg_ratios(df, powers, channels):    
    cols = df.columns
    inds = df.index
    sudep_eeg_ratios = {}
    c1_eeg_ratios = {}
    c2_eeg_ratios = {}
    rows, cols, res_cols = get_channel_names(powers, channels)
    for subject in inds:
        if df['asleep'][subject].shape != (1, 0) and df['awake'][subject].shape != (1, 0):
            asleep = df['asleep'][subject][rows, cols].flatten()
            awake = df['awake'][subject][rows, cols].flatten()
            sudep_eeg_ratios[subject] = asleep/awake
            
        if df['C1_asleep'][subject].shape != (1, 0) and df['C1_awake'][subject].shape != (1, 0):
            asleep = df['C1_asleep'][subject][rows, cols].flatten()
            awake = df['C1_awake'][subject][rows, cols].flatten()
            c1_eeg_ratios[subject+"C1"] = asleep/awake
            
        if df['C2_asleep'][subject].shape != (1, 0) and df['C2_awake'][subject].shape != (1, 0):
            asleep = df['C2_asleep'][subject][rows, cols].flatten()
            awake = df['C2_awake'][subject][rows, cols].flatten()
            c2_eeg_ratios[subject+"C2"] = asleep/awake
            
            
    return pd.concat([pd.DataFrame(sudep_eeg_ratios, index=res_cols).transpose(), \
           pd.DataFrame(c1_eeg_ratios, index=res_cols).transpose(), \
           pd.DataFrame(c2_eeg_ratios, index=res_cols).transpose()], axis=0)

In [ ]:
def get_eeg_ratios2(df, powers, channels):    
    cols = df.columns
    inds = df.index
    sudep_eeg_ratios = {}
    c1_eeg_ratios = {}
    c2_eeg_ratios = {}
    rows, cols, res_cols = get_channel_names(powers, channels)
    for subject in inds:
        if df['asleep'][subject].shape != (1, 0):
            sudep_eeg_ratios[subject] = df['asleep'][subject][rows, cols].flatten()
            
        if df['C1_asleep'][subject].shape != (1, 0):
            c1_eeg_ratios[subject+"C1"] = df['C1_asleep'][subject][rows, cols].flatten()
            
        if df['C2_asleep'][subject].shape != (1, 0):
            c2_eeg_ratios[subject+"C2"] = df['C2_asleep'][subject][rows, cols].flatten()
            
            
    return pd.concat([pd.DataFrame(sudep_eeg_ratios, index=res_cols).transpose(), \
           pd.DataFrame(c1_eeg_ratios, index=res_cols).transpose(), \
           pd.DataFrame(c2_eeg_ratios, index=res_cols).transpose()], axis=0)

In [ ]:
def get_channel_names(powers, channels):
    res_cols = []
    rows = []
    cols = []
    for power, channel in zip(powers, channels):
        if channel == "all":
            rows += [i for i in range(9)]
            cols += [power]*9
            res_cols += [power_to_string[power]+" "+str(c+1) for c in range(9)]
        else:
            rows += channel
            cols += [power]*len(channel)
            res_cols += [power_to_string[power]+" "+str(c+1) for c in channel]
    return rows, cols, res_cols

In [ ]:
def get_subject(name):
    lower = name.lower()
    ind = max(lower.find("awake"), lower.find("asleep")) - 1
    return name[:ind]

In [ ]:
AWAKE = "awake"
ASLEEP = "asleep"

def get_state(name):
    if "awake" in name.lower():
        return AWAKE
    else:
        return ASLEEP

def get_ecg_ratios(df, add = ""):
    inds = df.index
    d = {}
    ratios = {}
    res = []
    for i in range(inds.shape[0]):
        name = get_subject(inds[i])
        state = get_state(inds[i])
        
        if name in d:
            if state == AWAKE:
                ratios[name] = d[name]/df.iloc[i, :]
            else:
                ratios[name] = df.iloc[i, :]/d[name]
        else:
            d[name]=df.iloc[i, :]
    return pd.DataFrame(ratios, index=df.columns).transpose()

def get_ecg_ratios2(df, add = ""):
    inds = df.index
    d = {}
    ratios = {}
    res = []
    for i in range(inds.shape[0]):
        name = get_subject(inds[i])
        state = get_state(inds[i])
        
        if state == ASLEEP:
            ratios[name] = df.iloc[i,:]

    return pd.DataFrame(ratios, index=df.columns).transpose()

In [ ]:
def combine(X, Y, eeg, ecg, names):
    res_x, res_y, res_ekg_ecg, included = [], [], [], []
    eeg_names = set(eeg.index)
    ecg_names = set(ecg.index)
    assert(len(X) == len(names))
    
    for i in range(len(names)):
        name = names[i]
        name = name[:name.lower().rfind(' a')]
        if name not in eeg_names or name not in ecg_names:
            continue
        res_x.append(X[i])
        res_y.append(Y[i])
        res_ekg_ecg.append(np.concatenate([eeg.loc[name], ecg.loc[name]]))
        included.append(names[i])
    return res_x, res_y, res_ekg_ecg, included

In [ ]:
def show_amps():
    samps = []
    camps = []
    for i in range(len(X_train)):
        amp = X_train[i].max()
        if y_train[i]:
            samps.append(amp)
        else:
            camps.append(amp)
    plt.hist(samps, label="mean {}".format(np.mean(samps)))
    plt.legend()
    plt.show()
    
    plt.hist(camps, label="mean {}".format(np.mean(camps)))
    plt.legend()
    plt.show()
    
    samps = []
    camps = []
    for i in range(len(X_train)):
        amp = X_train[i].std()
        if y_train[i]:
            samps.append(amp)
        else:
            camps.append(amp)
    plt.hist(samps, label="std {}".format(np.mean(samps)))
    plt.legend()
    plt.show()
    
    plt.hist(camps, label="std {}".format(np.mean(camps)))
    plt.legend()
    plt.show()

In [ ]:
eeg_dir = './EEG Files - Zhe Chen'
paths = get_paths(eeg_dir)

chan_file = 'channel_dict_more.txt'

with open(chan_file,'r') as inf:
    d = eval(inf.read())
    
keys = list(d.keys())

if "more" not in chan_file:
    for key in keys:
        d[key[:key.rfind(' (')]+".txt"] = d.pop(key)

In [ ]:
delta = 0
theta = 1
alpha = 2
beta = 3
low_gamma = 4
high_gamma = 5

power_to_string = {
    delta : "delta",
    theta : "theta",
    alpha  :"alpha",
    beta  :"beta",
    low_gamma : "low_gamma",
    high_gamma  :"high_gamma"
}

In [ ]:
f =  200

X_train, y_train = [], []
X_test, y_test = [], []
included_train = []
included_test = []

test_inst = "melboaurne"

for path in paths:
    if "awake" in path.lower():
        continue
    if test_inst not in path.lower():
        X, Y, I = get_data("", path, f)
        X_train += X
        y_train += Y
        included_train += I
#     else:
#         X_test += X
#         y_test += Y
#         included_test += I
    
clear_output()

print('There are {} subjects'.format(len(included_train)+len(included_test)))
print("Training Set")
for i in included_train: print(i)
    
print("\nTesting Set")
for i in included_test: print(i)

In [ ]:
# for i in range(len(included_train)):
#     subject = included_train[i]
#     x = X_train[i]
#     x[np.isnan(x)] = np.median(x[~np.isnan(x)])
#     bound = x.mean()+4*x.std()
#     X_train[i][x > bound] = bound
#     X_train[i][x < -bound] = -bound
#     sub = subject[:min(subject.find(' '), subject.find('-') if subject.find('-') > 0 else 10000000, subject.find('_') if subject.find('_') > 0 else 10000000)]
#     if "C1dd" in subject:
#         c1[sub].append(x.max())
#     elif "C2" in subject:
#         c2[sub].append(x.max())
#     else:
#         s[sub].append(x.max())

In [ ]:
# file_dir='.\\ekg_features\\'

# drop = ['nni_50', 'pnni_50', 'nni_20', 'pnni_20', 'sampen']

# sudep = pd.read_excel(file_dir+"SUDEP.xlsx", index_col=[0], engine='openpyxl').drop(drop, axis=1)
# c1 = pd.read_excel(file_dir+"Control_1.xlsx", index_col=[0], engine='openpyxl').drop(drop, axis=1)
# c2 = pd.read_excel(file_dir+"Control_2.xlsx", index_col=[0], engine='openpyxl').drop(drop, axis=1)

# mean_p_area = loadmat('.\\mat_files\\mean_p_area.mat')['mean_p_area']
# for i in range(1,  mean_p_area.shape[0]):
#     mean_p_area[i][0] = mean_p_area[i][0][0]
# for i in range(1,  mean_p_area.shape[1]):
#     mean_p_area[0][i] = mean_p_area[0][i][0]
    
# index = mean_p_area[1:, 0]
# columns = mean_p_area[0, 1:]

# mean_p_area = pd.DataFrame(mean_p_area[1:, 1:], columns=columns, index=index)

# powers = []
# channels = []
# if True:
#     powers = [alpha, low_gamma]
# #     channels = [[1], [6, 8]]
#     channels = [[1], [6]]
# eeg = get_eeg_ratios(mean_p_area, powers, channels)

# ecg_feats = []
# if True:
# #     ecg_feats = ['lfnu', 'hfnu', 'ratio_sd2_sd1', 'sd1']
#     ecg_feats = ['lfnu']
# ecg = pd.concat([get_ecg_ratios(sudep)[ecg_feats],
#                  get_ecg_ratios(c1, "C1")[ecg_feats],
#                  get_ecg_ratios(c2, "C2")[ecg_feats]])

# X_train, y_train, ekg_eeg_train, included_train = combine(X_train, y_train, eeg, ecg, included_train)
# X_test, y_test, ekg_eeg_test, included_test = combine(X_test, y_test, eeg, ecg, included_test)

# print('There are {} subjects'.format(len(included_train)+len(included_test)))
# print("Training Set")
# included_train.sort()
# for i in included_train: print(i)
    
# print("\nTesting Set")
# for i in included_test: print(i)

In [ ]:
def join(X, Y, names, i, j):
    inds = sorted(i+j, reverse=True)
    for a in inds:
        X.pop(a)
        Y.pop(a)
        names.pop(a)
    
    n = len(X)
    for _ in range(0):
        for i in range(n):
            x = X[i]
            y = Y[i]
            if y[0][0] == 1:
                X.append(x)
                Y.append(y)
    
    
    res_X = np.concatenate(X)
    res_Y = np.concatenate(Y)
        
    return res_X, res_Y

In [ ]:
def my_roc(true, pred, points=51):
    true = np.asarray(true)
    pred = np.asarray(pred)
    thresholds = np.linspace(0, 1, num=points, endpoint=True)
    tprs = []
    fprs = []
    tps, tns, fps, fns = [], [], [], []
    for i in range(thresholds.shape[0]):
        t = thresholds[i]
        t_pred = pred > t
        tp = np.logical_and(t_pred==1, true==1).sum()
        fp = np.logical_and(t_pred==1, true==0).sum()
        tn = np.logical_and(t_pred==0, true==0).sum()
        fn = np.logical_and(t_pred==0, true==1).sum()
        
        tprs.append(tp/(tp+fn))
        fprs.append(fp/(tn+fp))
        tps.append(tp)
        tns.append(tn)
        fps.append(fp)
        fns.append(fn)
        
    res = {
        'thresholds':thresholds,
        'tprs':tprs,
        'fprs':fprs,
        'tps':tps,
        'tns':tns,
        'fps':fps,
        'fns':fns
    }
    
    for key, val in res.items():
        res[key] = np.array(val)

    return res

In [ ]:
def my_auc(true, pred):
    res = my_roc(true, pred)
    return integrate(res['fprs'], res['tprs'])

In [ ]:
def my_loss(true, pred):
    true =np.array(true)
    pred = np.array(pred)
    t = options['zero_thresh']
    pred[pred<t] = t
    pred[pred>(1-t)] = 1-t
    
    return log_loss(true, pred)

In [ ]:
def two_plots(loss, auc):
    fig, ax1 = plt.subplots()
    
    color='tab:red'
    ax1.set_ylabel('loss', color=color)
    ax1.plot(loss, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()
    
    color='tab:blue'
    ax2.set_ylabel('AUC', color=color)
    ax2.plot(auc)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.axis(ymin=-0.1, ymax=1.1)
    
    plt.show()

In [ ]:
def val_split(split=0.5, time=800):
    train_x, train_y, val_x, val_y = [], [], [], []
    
    X, Y = [], []
    
    names = []
    for i in pick_balanced(y_train):
        x, y = data_X[i][0], data_Y[i]
        X.append(x)
        Y.append(y)
        names.append(included_train[i])

    for i in range(len(X)):
        x, y = X[i], Y[i]
        ind = int(len(x)*split)
        train_x.append(x[:ind])
        val_x.append(x[ind:])
        train_y.append(y[:ind])
        val_y.append(y[ind:])
    print(train_x[0].shape)
    print(x.shape)
    return np.concatenate(train_x), np.concatenate(train_y), val_x, val_y, names

In [ ]:
def train_val_test_inds(y):
    inds = [i for i in range(len(y))]
    if options['balanced']:
        inds = [inds[i] for i in pick_balanced(y)]
        y = [y[i] for i in inds]
    train_x, val_x, train_y, val_y = train_test_split(inds, y, 
                                                      test_size = options['val_split'], stratify=y)
    
    
    
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, 
                                                        test_size=options['test_split']/(1-options['val_split']),
                                                        stratify = train_y)
    test_x = [test_x[i] for i in pick_balanced(test_y)]
    
    return train_x, val_x, test_x

In [ ]:
def helper(train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features):
    X_train_np = np.vstack(train_x)
    y_train_np = np.vstack(train_y)
    e_train_np = np.vstack(train_features)
    
    min_loss = float('inf')
    
    model = create_model(combo[1], 9, train_features[0].shape[1])
    
    p = 0
    
    true, pred = [], []
    
    for i in range(1, options['epochs']+1):
        inds = np.random.choice(X_train_np.shape[0], size=batch_size)
        model.fit([X_train_np, e_train_np], y_train_np, verbose=1)
        if i % options['valid'] == 0:
            curr_val_loss, _, _, _ = test(val_x, val_y, val_features, model)
            print("val_loss", curr_val_loss)
            if curr_val_loss < min_loss:
                min_loss = curr_val_loss
                _, _, true, pred = test(test_x, test_y, test_features, model)
                p = 0
            elif p >= options['patience']:
                break
            else:
                p += 1
                
    return true, pred

In [ ]:
def save_roc(data, test, folder):
    save_name=folder

    fontsize={
        'title':50,
        'axis':50,
        'ticks':40,
        'legend':40
    }
    
    copy = {}

    for key, val in data.items():
        copy[key] = [val[i].tolist() for i in range(len(val))]
        
    with open(f'.\ROC\\{save_name}\\{test} results {combo[0]}-{combo[1]}.txt', 'w') as f:
        print(copy, file=f)

    tprs = np.array(data['tprs'])
    for key, val in copy.items():
        copy[key] = np.array(val).mean(axis=0)

    mean_fpr = copy['fprs']
    mean_tpr = copy['tprs']

    plt.figure(figsize=(15,12))

    plt.tick_params(labelsize=fontsize['ticks'])
    plt.title("ROC Syncnet", fontsize=fontsize['title'])
    plt.xlabel('False Positive Rate', fontsize=fontsize['axis'])
    plt.ylabel('True Positive Rate', fontsize=fontsize['axis'])

    area = integrate(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, label="Area: {:.3f}".format(area), linewidth=3)
    plt.plot([0, 1], [0, 1], linestyle='dashed')

    CIs = CI(tprs)
    #plt.vlines(x=mean_fpr, ymin=CIs[:, 0], ymax=CIs[:, 1], color="black", linewidth=3)
    #plt.hlines(y=CIs[:, 0], xmin=mean_fpr-width, xmax=mean_fpr+width, color="black", linewidth=3)
    #plt.hlines(y=CIs[:, 1], xmin=mean_fpr-width, xmax=mean_fpr+width, color="black", linewidth=3)
    plt.fill_between(mean_fpr, CIs[:, 0], CIs[:, 1], alpha=0.2)
    plt.legend(loc='upper left', fontsize=fontsize['legend'])
    data["lower"] = CIs[:, 0]
    data["upper"] = CIs[:, 1]
    pd.DataFrame(copy).to_excel(f".\ROC\\{save_name}\\Confidence Interval, {combo[0]}-{combo[1]}, {test}.xlsx", index=False)

    plt.savefig(f".\ROC\\{save_name}\\ROC Curve {combo[0]}-{combo[1]}, {test}.jpg")

In [ ]:
import scipy.stats as st

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
def test(X, Y, F, model):
    true, pred = [], []
    for x, y, f in zip(X, Y, F):
        p = model.predict(x+[f])
        pred.append(np.mean(p))
        true.append(y[0][0])
    return my_loss(true, pred), my_auc(true, pred), true, pred

def actual_test(X, Y, F, model):
    true = np.concatenate(Y)
    pred = model.predict([np.concatenate([x[i] for x in X]) for i in range(len(X[0]))]+[np.vstack(F)])
    return my_loss(true, pred), my_auc(true, pred), true, pred

def MC_test(X, Y, F, model):
    with tf.keras.backend.learning_phase_scope(1):
        pred = np.stack([[model.predict([x, f]).mean() for x, f in zip(X, F)] for _ in range(options['MC_num_iters'])])
    return [y[0][0] for y in Y], list(pred.mean(axis=0))

In [ ]:
def one_run(ekg_eeg_feats):
    train, val, testi = train_val_test_inds(y_train)
    train_x, train_y, val_x, val_y, test_x, test_y = [], [], [], [], [], []
    train_features, val_features, test_features = [], [], []
    
    print("training:")
    for i in train:
#         x, y = to_train_samples(X_train, y_train, [i], options['units'])
        x, y = data_X[i], data_Y[i]
        train_x.append(x)
        train_y.append(y)
        train_features.append(np.vstack([ekg_eeg_feats[i] for _ in range(x[0].shape[0])]))
        
        print(included_train[i][:-4])
    print()
    print("validation:")
    for i in val:
#         x, y = to_train_samples(X_train, y_train, [i], options['units'])
        x, y = data_X[i], data_Y[i]
        val_x.append(x)
        val_y.append(y)
        val_features.append(np.vstack([ekg_eeg_feats[i] for _ in range(x[0].shape[0])]))
        
        print(included_train[i][:-4])
    print()
        
    test_names = []
    print("testing:")
    for i in testi:
#         x, y = to_train_samples(X_train, y_train, [i], options['units'])
        x, y = data_X[i], data_Y[i]
        test_x.append(x)
        test_y.append(y)
        test_features.append(np.vstack([ekg_eeg_feats[i] for _ in range(x[0].shape[0])]))
        test_names.append(included_train[i])
        
        print(included_train[i][:-4])
    print()
        
    num_repeated = 0
    
    all_info = {
        "names": [],
        "prediction": [],
        "actual": []
    }
    model = None 
    while True:
        try:
            tf.keras.backend.clear_session()
            callback = tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=0,
                    patience=options['patience'],
                    verbose=0,
                    mode="auto",
                    baseline=None,
                    restore_best_weights=True,
                )
            model = create_model(options['C'], 0)
#                 X_train_np = np.concatenate(train_x)
#                 X_val_np = np.concatenate(val_x)
            model.fit([np.concatenate([x[i] for x in train_x]) for i in range(len(train_x[0]))]+[np.vstack(train_features)], 
                  np.concatenate(train_y), 
                  epochs=options['epochs'],
                  batch_size=options['batch_size'],
                  validation_data=([np.concatenate([x[i] for x in val_x]) for i in range(len(val_x[0]))]+[np.vstack(val_features)], np.concatenate(val_y)),
                  verbose=options['verbose'],
                  callbacks=[callback],
                  class_weight=options['class_weight'])
            val_auc = actual_test(val_x, val_y, val_features, model)[1]
            if val_auc == 0.5 and num_repeated < 10:
                num_repeated += 1
                print("repeat", num_repeated)
                continue
            break
        except KeyboardInterrupt:
            return None, None
        except Exception as e:
            print(e)
            del model
            continue



    _,_,t,p = test(test_x, test_y, test_features, model)
#         t, p = ensemble(train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features, test_names)
    test_auc = roc_auc_score(t, p)

    all_info['names'].append(test_names)
    all_info['prediction'].append(p)
    all_info['actual'].append(t)
    
    return None, test_auc, all_info, model

In [ ]:
def n_fold_one_run(train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features):
    num_repeated = 0
        
    while True:
        try:
            tf.keras.backend.clear_session()
            callback = tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=0,
                    patience=options['patience'],
                    verbose=0,
                    mode="auto",
                    baseline=None,
                    restore_best_weights=True,
                )
            model = create_model(options['C'], train_features[0].shape[1])
#                 X_train_np = np.concatenate(train_x)
#                 X_val_np = np.concatenate(val_x)
            model.fit([np.concatenate([x[i] for x in train_x]) for i in range(len(train_x[0]))]+[np.vstack(train_features)], 
                  np.concatenate(train_y), 
                  epochs=options['epochs'],
                  batch_size=options['batch_size'],
                  validation_data=([np.concatenate([x[i] for x in val_x]) for i in range(len(val_x[0]))]+[np.vstack(val_features)], np.concatenate(val_y)),
                  verbose=options['verbose'],
                  callbacks=[callback])
            val_auc = actual_test(val_x, val_y, val_features, model)[1]
            if val_auc == 0.5 and num_repeated < 10:
                num_repeated += 1
                print("repeat", num_repeated)
                continue
            break
        except KeyboardInterrupt:
            return None, None
        except Exception as e:
            print(e)
            del model
            continue
    _,_,t,p = test(test_x, test_y, test_features, model)
    print(model.layers[-1].get_weights())
    del model
    return t,p

def split(X, Y, names, data_features, train_index, test_index):
    train_x = [X[i] for i in train_index]
    train_y = [Y[i] for i in train_index]
    data_features = np.array(data_features)
#     if data_features.shape[1] != 0:
#         scaler = MinMaxScaler(feature_range=(options['min'],options['max']))
#         data_features[train_index] = scaler.fit_transform(data_features[train_index])
#         data_features[test_index] = scaler.transform(data_features[test_index])
    train_features = [np.vstack([data_features[i] for _ in range(X[i][0].shape[0])]) for i in train_index]
    
    test_x = [X[i] for i in test_index]
    test_y = [Y[i] for i in test_index]
    test_features = [np.vstack([data_features[i] for _ in range(X[i][0].shape[0])]) for i in test_index]
    test_names = [names[i] for i in test_index]
    
    train_x, val_x, train_y, val_y, train_features, val_features = train_test_split(train_x, train_y, train_features, test_size = options['val_split'], stratify=[y[0][0] for y in train_y])
    
    return train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features, test_names

def nfold(ekg_eeg_feats, folds=5, fold_num=0):
    
    def my_test(model):
        pred = []
        true = []
        
        for x, y, f in zip(test_x, test_y, test_features):
            p = model.predict([x[:,i,:,:] for i in range(x.shape[1])]+[f]).mean()
            pred.append(p)
        return [y[0][0] for y in test_y], pred
    
    skf = StratifiedKFold(shuffle=True, n_splits=folds)
    
    all_info = {
        "names": [],
        "prediction": [],
        "actual": []
    }
    
    X, Y, data_features, names = [], [], [], []
    asd = []
    min_samples = 20000
    for i in pick_balanced(y_train):
        x, y = data_X[i], data_Y[i]
#         x, y = to_train_samples(X_train, y_train, [i], options['units'])
        min_samples = min(x[0].shape[0], min_samples)
        X.append(x)
        Y.append(y)
        data_features.append(ekg_eeg_feats[i])
        names.append(included_train[i])
        asd.append(y[0][0])
    
#     for i in range(len(X)):
#         inds = np.random.choice(X[i][0].shape[0], min_samples, replace=False)
#         X[i] = [x[inds] for x in X[i]]
#         Y[i] = Y[i][inds]
    
    dummy = [i for i in range(len(asd))]
    i = 1
    true, pred = [], []
    
    auc_sum = 0
    
#     for train_index, test_index in five_fold_ind(predefined_folds[fold_num]):
    for train_index, test_index in skf.split(dummy, asd):
        print(f"fold {i}/{folds}")
        i+=1
        train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features, test_names = split(X, Y, names, data_features, train_index, test_index)        
        t,p = n_fold_one_run(train_x, train_y, val_x, val_y, test_x, test_y, train_features, val_features, test_features)
        test_auc = roc_auc_score(t, p)
        print(t)
        print(p)
        auc_sum += test_auc
        print(test_auc, auc_sum/(i-1))
        
        true += t
        pred += p

        all_info['names'].append(test_names)
        all_info['prediction'].append(p)
        all_info['actual'].append(t)
    return my_roc(true, pred), auc_sum / folds, all_info
        

In [ ]:
def create_model(channels, num_features):
    features_input = tf.keras.layers.Input(shape=(num_features,), name="features_input")
    all_inputs = []
    all_logits = []
    
    for i in range(len(options['freqs'])):    
        raw_eeg_input = tf.keras.layers.Input(shape=(options['d_units'][i], channels), name="raw_eeg_input_{}".format(i+1))
        all_inputs.append(raw_eeg_input)
        raw_eeg_input = tf.keras.layers.SpatialDropout1D(options['spatial_dropout'])(raw_eeg_input)
        syncnet_layer = SyncNetLayer(options, options['filter_sizes'][i], i)(raw_eeg_input)
        logits = tf.keras.layers.Flatten()(syncnet_layer)
#         logits = tf.keras.layers.ReLU(max_value=None)(logits)
#         logits  = tf.keras.layers.Lambda(lambda x: 5*(1/(1+tf.exp(-0.1*x))))(logits)
#         logits = tf.keras.layers.BatchNormalization()(logits)
        all_logits.append(logits)
    
    if num_features!=0:
        all_inputs.append(features_input)
        all_logits.append(features_input)
#     print(all_logits)
    if len(all_logits) > 1:
        concat = tf.keras.layers.Concatenate(axis=1)(all_logits)
    else:
        concat = all_logits[0]
#     concat = tf.keras.layers.BatchNormalization()(concat)
    dropout = tf.keras.layers.Dropout(options['dropout'])(concat)
    
#     dropout = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(options['lambda']))(dropout)
#     dropout = tf.keras.layers.Dropout(options['dropout'])(dropout)
    
#     dropout = tf.keras.layers.Dense(4, activation='selu',kernel_regularizer=tf.keras.regularizers.l2(options['lambda']))(dropout)
#     dropout = tf.keras.layers.Dropout(options['dropout'])(dropout)
    
    out = tf.keras.layers.Dense(1, activation='sigmoid',
                                kernel_regularizer=tf.keras.regularizers.l2(options['lambda']))(dropout)
    model = tf.keras.Model(inputs=all_inputs, outputs=out, name="SyncNet")
    
#     options['lr'] = tf.keras.optimizers.schedules.ExponentialDecay(
#                                         initial_learning_rate=0.5,
#                                         decay_steps=10000,
#                                         decay_rate=0.99)


    options['optimizer'] = tf.keras.optimizers.Adam(options['lr'])
    
    model.compile(loss="binary_crossentropy", optimizer=options['optimizer'], metrics=['AUC'])
    return model
    

In [ ]:
options = {}

In [ ]:
window = 400

options['offset'] = window
options['select'] = False
options['units'] = window
options['d_units'] = [window]
options['normalize'] = False
options['min_max'] = False
options['max'] = 1
options['min'] = 0
options['smooth'] = 0
options['order'] = 0
options['freqs'] = [200]

data_X, data_Y = [], []

for i in range(len(X_train)):
    print(f"{i+1}/{len(X_train)}")
    x, y = to_train_samples(X_train, y_train, [i], options['units'])
    data_X.append(x)
    data_Y.append(y)
clear_output()

In [ ]:
ekg_eeg_train = [[] for _ in range(len(X_train))]

In [ ]:
def split_test():
    
    def test(X, Y):
        true, pred = [], []
        for x, y in zip(X, Y):
#             print(x.shape)
            p = model.predict(x)
            pred.append(p.mean())
            true.append(y[0][0])
        return my_loss(true, pred), my_auc(true, pred), true, pred
    
    def actual_test(X, Y):
        true = np.concatenate(Y)
        pred = model.predict(np.concatenate(X))
        return my_loss(true, pred), my_auc(true, pred), true, pred
    
    
    x_train, y_train, x_val, y_val, names = val_split()
    model = create_model(options['C'], 0)
    
    callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                min_delta=0,
                patience=10,
                verbose=0,
                mode="auto",
                baseline=None,
                restore_best_weights=True,
               )

    model.fit(x_train, 
              y_train, 
              batch_size=options['batch_size'], 
              epochs=options['epochs'], 
              callbacks=[callback],
             verbose=options['verbose'],
             validation_split=0.1)
    
    _,_,true, pred = test(x_val, y_val)
    
    info = {
        "names":names,
        "prediction":pred,
        "actual":true
    }
    
    return None, roc_auc_score(true, pred), info

In [ ]:
included_feats = [[] for i in range(len(included_train))]

In [ ]:
#samples
options['epochs'] = 1000
options['num_labels'] = 1
options['batch_size'] = 128
#conv2d
options['C'] = X_train[0].shape[1]
options['K'] = 4
options['filter_sizes'] = [window//2]
options['pool_size'] = [window]
options['stride'] = [window]

#regularization
options['dropout'] = 0.5
options['spatial_dropout'] = 0.

options['lambda'] = 0.0
options['lambda_2'] = 0.0
options['lambda_b'] = 0
options['lambda_omega'] =0
options['lambda_beta'] = 0
options['lambda_phi'] =0
options['lr'] = 0.1

options['stddev'] = 0.0
options['MC_num_iters'] = 20
options['early_stop'] = True
options['patience'] = 10
options['valid'] = 1

#idk
options['upper_omega']= [50, 50]
options['lower_omega'] = [0, 0]
options['class_weight'] = {0:1.0, 1:1.0}
# options['lr'] = 0.005

options['freq'] = f

#data sets
options['test_split'] = .2
options['val_split'] = 0.1

options['zero_thresh'] = 1e-5

options['select'] = False
options['beta1'] = 0.9 #default is 0.9
options['beta2'] = 0.999 #default is 0.999
options['balanced'] = False
options['verbose'] = 1



num_trials = 100
# for feats, save_name in zip([(0,0),(0,3)], ['Raw Data', 'Raw Data + EEG Feats']):
for feats, save_name in zip([(0,0)], ['Raw Data']):
# for feats, save_name in zip([(0,3)], ['Raw Data + EEG Feats']):
# for feats, save_name in zip([(0,3)], ['Raw Data + EEG + EKG Feats']):
    all_infos_arr = []
    auc = []
    
    included_feats = [a[feats[0]:feats[1]] for a in ekg_eeg_train]
    i = len(all_infos_arr)
    while i < num_trials:
        print(i+1)
        try:
#             res, res_auc, info = one_run_two(included_feats)
#             res, res_auc, info = one_run(included_feats)
            res, res_auc, info = nfold(included_feats, 5, i)
#             res, res_auc, info = split_test()
        except KeyboardInterrupt:
            break
#         except Exception as e:
#             print(e)
#             continue

        clear_output()
#         auc.append(integrate(res['fprs'], res['tprs']))
        auc.append(res_auc)
        print(mean_confidence_interval(auc))
        print(np.median(auc))
        print(res_auc)
        plt.hist(auc)
        plt.show()
        all_infos_arr.append(info)
        i += 1
#         with open('.\ROC\\5-fold Cross Validation Final\\{} Results 8s other.txt'.format(save_name), 'w') as g:
#             print(all_infos_arr, file=g)
#     if len(all_infos_arr) == num_trials:
#         with open('.\ROC\\5-fold Cross Validation Final\\{} Results 8s other.txt'.format(save_name), 'w') as g:
#             print(all_infos_arr, file=g)
#     save_roc(data, save_name)
    plt.clf()